[Poprzedni rozdzial](rozdzial_04_ustawienia_eksperymentow.ipynb)


# 5. Wyniki

In [ ]:
from pathlib import Path
import pickle
from IPython.display import HTML, display

ROOT = Path.cwd().resolve()
while not (ROOT / "parameters.py").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

def _html_table(table):
    header = "".join([f"<th>{h}</th>" for h in table[0]])
    rows = []
    for row in table[1:]:
        cells = "".join([f"<td>{c}</td>" for c in row])
        rows.append(f"<tr>{cells}</tr>")
    table_html = "<table>" + "<tr>" + header + "</tr>" + "".join(rows) + "</table>"
    return table_html

PLOTS_DIR = ROOT / "plots"


W analizie przyjęto ustalone wcześniej parametry modelu i opcji $S_0$, $K$, $r$, $\sigma$ oraz $T$. Wszystkie estymatory porównywane są przy tym samym budżecie symulacji $R = 10^5$ trajektorii.

## 5.1. Opcja europejska ($C=\infty,\,n=1$)

W niniejszej części analizy zajmujemy się estymacją ceny europejskiej opcji kupna. Porównujemy tu wszystkie estymatory, analizując ich dokładność i wariancję. Jako punkt odniesienia wykorzystujemy teoretyczną wartość opcji daną wzorem Blacka–Scholesa, która w tym przypadku jest znana w postaci jawnej. Dzięki temu możliwe jest bezpośrednie porównanie estymatorów oraz ocena ich błędu i efektywności numerycznej.

### 5.1.1. Tabela

Tworzymy zbiorczą tabelę porównującą wszystkie rozważane estymatory ceny europejskiej opcji kupna. Zawiera ona średnią estymatę, odchylenie standardowe, obciążenie, błąd średniokwadratowy oraz empiryczne pokrycie 95% przedziału ufności, co pozwala bezpośrednio ocenić ich dokładność i efektywność.

In [8]:
stats_path = ROOT / "data" / "european_estimator_stats.pkl"
with stats_path.open("rb") as f:
    payload = pickle.load(f)

table = payload.get("display_table")
display(HTML(_html_table(table)))

METHOD,MEAN,SD,BIAS,MSE,CI-COVER-95%
crude,16.263,0.088,-0.278 · 10-3,0.008,.945
stratified (proportional),16.260,0.074,-0.365 · 10-2,0.005,.974
stratified (optimal),16.263,0.058,-0.142 · 10-3,0.003,.946
antitetic,16.263,0.068,-0.697 · 10-4,0.005,.987
control variate,16.263,0.049,-0.582 · 10-4,0.002,.956


### 5.1.2. Empiryczny VRF

Empiryczną wariancję redukcji (VRF) pomiędzy dwiema metodami estymacji
$A$ oraz $B$ definiujemy jako
$$
\mathrm{VRF}(A,B)
=
\frac{\widehat{\mathrm{Var}}\!\left(\hat{I}^{(A)}\right)}
{\widehat{\mathrm{Var}}\!\left(\hat{I}^{(B)}\right)},
$$
gdzie $\hat{I}^{(A)}$ i $\hat{I}^{(B)}$ oznaczają estymatory uzyskane odpowiednio metodą $A$ i $B$.
Wartość $\mathrm{VRF}(A,B) > 1$ oznacza, że metoda $B$ charakteryzuje się mniejszą wariancją niż metoda $A$.

In [9]:
vrf_path = ROOT / "data" / "european_vrf_table.pkl"
with vrf_path.open("rb") as f:
    vrf_payload = pickle.load(f)

table = vrf_payload.get("display_table")
display(HTML(_html_table(table)))

VRF = VAR(ROW) / VAR(COL),Crude,Stratified (proportional),Stratified (optimal),Antithetic,Control variate
Crude,1,1.409,2.286,1.683,3.202
Stratified (proportional),-,1,1.623,1.195,2.273
Stratified (optimal),-,-,1,0.736,1.400
Antithetic,-,-,-,1,1.902
Control variate,-,-,-,-,1


### 5.1.3. Wykresy

Dla każdego estymatora przedstawiamy histogramy oraz wykresy pudełkowe, aby wizualnie porównać rozkłady otrzymanych estymat i ich zmienność.

In [10]:
hist_path = PLOTS_DIR / "european_errors_hist.pkl"
box_path = PLOTS_DIR / "european_errors_box.pkl"

with hist_path.open("rb") as f:
    hist_fig = pickle.load(f)
hist_fig

In [11]:
with box_path.open("rb") as f:
    box_fig = pickle.load(f)
box_fig

### 5.1.4. Analiza i interpretacja

Wszystkie zastosowane metody dały estymaty bardzo bliskie wartości teoretycznej, a obserwowane obciążenie było pomijalnie małe (rzędu $10^{-3}$–$10^{-4}$). Wskazuje to na poprawną implementację symulacji oraz funkcji wypłaty, a także na nieobciążoność rozważanych estymatorów.

Zastosowanie technik redukcji wariancji przyniosło wyraźne korzyści w porównaniu z metodą crude Monte Carlo. Najmniejszą wariancją i błędem średniokwadratowym charakteryzował się estymator ze zmienną kontrolną, co jest zgodne z teorią, gdyż zmienna kontrolna $B(1)$ jest silnie skorelowana z wypłatą europejskiej opcji call. Próbkowanie warstwowe również prowadziło do istotnej redukcji wariancji, przy czym alokacja optymalna była wyraźnie skuteczniejsza od alokacji proporcjonalnej. Metoda antytetyczna dała umiarkowaną, lecz zauważalną poprawę względem estymatora naiwniego.

Empiryczne pokrycie przedziałów ufności na poziomie $95\%$ było bliskie wartości nominalnej dla wszystkich metod, co potwierdza poprawność estymacji wariancji. Rozkłady błędów estymacji były symetryczne i skoncentrowane wokół zera, a ich rozrzut malał wraz z zastosowaniem bardziej zaawansowanych technik redukcji wariancji.

Uzyskane wyniki dla opcji europejskiej stanowią punkt odniesienia dla dalszej analizy opcji barierowej, gdzie brak wzoru analitycznego sprawia, że efektywność metod Monte Carlo oraz technik redukcji wariancji odgrywa jeszcze istotniejszą rolę.


## 5.2. Opcja typu *up-and-out* ($C\in\{105, 120\},n=12$)

Analizujemy opcję barierową typu up-and-out. Rozważamy dwie wartości bariery: barierę bliską $C_1=105$ oraz barierę daleką $C_2=120$. Liczba kroków dyskretyzacji wynosi $n = 12$. Wszystkie estymatory porównywane są przy tym samym budżecie symulacji $R = 10^5$ trajektorii. W analizie uwzględniono trzy estymatory: standardowy estymator crude Monte Carlo oraz dwa estymatory stratified Monte Carlo — z alokacją proporcjonalną i z alokacją optymalną (Neymana).

### 5.2.1. Benchmark przez wartość referencyjną

Dla rozważanej opcji barierowej przy $n = 12$ nie jest dostępna zamknięta postać
analityczna ceny, co uniemożliwia bezpośrednie wyznaczenie dokładnej wartości
oczekiwanej payoffu oraz rzeczywistego błędu estymatorów Monte Carlo. W celu
umożliwienia ilościowego porównania wyników wprowadzona zostaje wartość
referencyjna $I_{\text{ref}}$.

Wartość $I_{\text{ref}}$ wyznaczana jest jako estymata Monte Carlo obliczona
z bardzo dużym budżetem symulacji $R_{\text{ref}} \gg R$, dzięki czemu jej błąd
statystyczny jest zaniedbywalnie mały w porównaniu z analizowanymi estymatorami.
W niniejszej pracy przyjęto
$$
R_{\text{ref}} = 10^{10}.
$$
Jako benchmark zastosowano estymator typu *crude Monte Carlo*, ze względu na jego
prostotę, bezstronność oraz niezależność od dodatkowych założeń konstrukcyjnych,
takich jak stratyfikacja czy alokacja próbek.

Należy podkreślić, że $I_{\text{ref}}$ nie stanowi dokładnej ceny teoretycznej,
lecz wysokiej precyzji przybliżenie, wykorzystywane wyłącznie jako punkt
odniesienia w analizie porównawczej.

### 5.2.2. Tabele wyników

Analogicznie do przypadku opcji europejskiej, przedstawiamy tabele statystyk opisowych dla estymacji opcji z barierą, co pozwala na bezpośrednie porównanie własności poszczególnych metod.

In [12]:
stats_path = ROOT / "data" / "barrier_estimator_stats.pkl"
with stats_path.open("rb") as f:
    payload = pickle.load(f)

tables = payload.get("display_tables", {})
for barrier, table in sorted(tables.items()):
    print(f"C = {barrier}")
    display(HTML(_html_table(table)))

C = 105.0


METHOD,MEAN,SD,BIAS,MSE,CI-COVER-95%
crude,0.019,0.744 · 10-3,0.107 · 10-3,0.565 · 10-6,.940
stratified (proportional),0.019,0.754 · 10-3,0.103 · 10-3,0.579 · 10-6,.943
stratified (optimal),0.018,0.214 · 10-1,-0.424 · 10-3,0.459 · 10-3,.197


C = 120.0


METHOD,MEAN,SD,BIAS,MSE,CI-COVER-95%
crude,0.668,0.828 · 10-2,-0.663 · 10-5,0.686 · 10-4,.946
stratified (proportional),0.668,0.809 · 10-2,0.998 · 10-4,0.655 · 10-4,.955
stratified (optimal),0.668,0.858 · 10-2,0.321 · 10-3,0.736 · 10-4,.949


### 5.2.3. Tabele redukcji wariancji

Podobnie jak wcześniej, dla opcji z barierą przedstawiamy tabelaryczne zestawienie empirycznych współczynników VRF dla wszystkich metod.

In [13]:
vrf_path = ROOT / "data" / "barrier_vrf_tables.pkl"
with vrf_path.open("rb") as f:
    vrf_payload = pickle.load(f)

for barrier, table in vrf_payload["tables"].items():
    print(f"C = {barrier}")
    display(HTML(_html_table(table.get("display_table"))))

C = 105.0


VRF = VAR(ROW) / VAR(COL),Crude,Stratified (proportional),Stratified (optimal)
Crude,1,0.974,0.001
Stratified (proportional),-,1,0.001
Stratified (optimal),-,-,1


C = 120.0


VRF = VAR(ROW) / VAR(COL),Crude,Stratified (proportional),Stratified (optimal)
Crude,1,1.047,0.932
Stratified (proportional),-,1,0.890
Stratified (optimal),-,-,1


### 5.2.4. Wykresy

Analogicznie do wcześniejszej analizy, dla opcji z barierą przedstawiamy histogramy oraz wykresy pudełkowe dla każdego estymatora.

In [15]:
hist_path = PLOTS_DIR / "barrier_errors_hist_C105.pkl"
box_path = PLOTS_DIR / "barrier_errors_box_C105.pkl"

with hist_path.open("rb") as f:
    hist_fig = pickle.load(f)
hist_fig

In [16]:
with box_path.open("rb") as f:
    box_fig = pickle.load(f)
box_fig

In [17]:
hist_path = PLOTS_DIR / "barrier_errors_hist_C120.pkl"
box_path = PLOTS_DIR / "barrier_errors_box_C120.pkl"

with box_path.open("rb") as f:
    box_fig = pickle.load(f)
hist_fig


In [18]:
with box_path.open("rb") as f:
    box_fig = pickle.load(f)
box_fig

### 5.2.5. Analiza i interpretacja

Dla bariery $C=120$, czyli relatywnie wysokiej w stosunku do ceny początkowej, zachowanie estymatorów było zbliżone do przypadku opcji europejskiej. Wszystkie metody dawały porównywalne wyniki, a różnice w wariancji pomiędzy estymatorem naiwnym oraz próbkowaniem warstwowym były niewielkie. Zarówno alokacja proporcjonalna, jak i optymalna nie prowadziły do istotnej redukcji wariancji, co potwierdzają współczynniki VRF bliskie jedności oraz podobne rozkłady błędów.

Sytuacja istotnie zmienia się dla bariery $C=105$, położonej blisko ceny początkowej aktywa. W tym przypadku estymator z próbkowaniem warstwowym i alokacją optymalną wykazywał bardzo niestabilne zachowanie: obserwowano silnie asymetryczny rozkład błędu, liczne obserwacje odstające oraz drastyczne pogorszenie pokrycia przedziałów ufności (około $20\%$ zamiast $95\%$). Wartość MSE dla tej metody była o kilka rzędów wielkości większa niż dla pozostałych estymatorów.

Zaobserwowane zjawisko wynika z faktu, że dla niskiej bariery większość trajektorii zostaje wyzerowana, a wypłata opcji jest silnie zależna od rzadkich zdarzeń, w których proces nie przekracza bariery i jednocześnie kończy się powyżej ceny wykonania. Alokacja optymalna, oparta na estymacji wariancji w warstwach, prowadzi w tym przypadku do nadmiernego skupienia próbek w wybranych stratach, co skutkuje niestabilnością estymatora.

Metoda crude Monte Carlo oraz próbkowanie warstwowe z alokacją proporcjonalną okazały się w tym przypadku znacznie bardziej stabilne. Ich wariancje były porównywalne, rozkłady błędów symetryczne, a empiryczne pokrycie przedziałów ufności bliskie wartości nominalnej $95\%$.

Uzyskane wyniki pokazują, że skuteczność technik redukcji wariancji w przypadku opcji barierowych silnie zależy od położenia bariery. Metody, które sprawdzają się bardzo dobrze dla opcji europejskich, mogą prowadzić do pogorszenia jakości estymacji w przypadku wypłat opartych na rzadkich zdarzeniach, zwłaszcza gdy bariera znajduje się blisko ceny początkowej instrumentu.

[Nastepny rozdzial](rozdzial_06_dyskusja_wnioski.ipynb)
